In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
dataCSV = [
    "/content/drive/MyDrive/project/ai_map/small-easy.csv",
    "/content/drive/MyDrive/project/ai_map/small-medium.csv",
    "/content/drive/MyDrive/project/ai_map/small-hard.csv",
    "/content/drive/MyDrive/project/ai_map/medium-easy.csv",
    "/content/drive/MyDrive/project/ai_map/medium-medium.csv",
    "/content/drive/MyDrive/project/ai_map/medium-hard.csv",
    "/content/drive/MyDrive/project/ai_map/big-easy.csv",
    "/content/drive/MyDrive/project/ai_map/big-medium.csv",
    # 無法無法100%有區域到不了
    "/content/drive/MyDrive/project/ai_map/big-hard.csv",
    "/content/drive/MyDrive/project/ai_map/small-easy.png",
    "/content/drive/MyDrive/project/ai_map/small-medium.png",
    "/content/drive/MyDrive/project/ai_map/small-hard.png",
    "/content/drive/MyDrive/project/ai_map/medium-easy.png",
    "/content/drive/MyDrive/project/ai_map/medium-medium.png",
    "/content/drive/MyDrive/project/ai_map/medium-hard.png",
    "/content/drive/MyDrive/project/ai_map/big-easy.png",
    "/content/drive/MyDrive/project/ai_map/big-medium.png",
    # 無法無法100%有區域到不了
    "/content/drive/MyDrive/project/ai_map/big-hard.png",
]

In [ ]:
import numpy as np
import pandas as pd
import math
import cv2
from google.colab import files


dirs = [
    lambda x, y: (x, y + 1),  # 右
    lambda x, y: (x + 1, y),  # 下
    lambda x, y: (x, y - 1),  # 左
    lambda x, y: (x - 1, y),  # 上
]

start=[]
end=[]
zeroList=[]
path=[]
pathSet = set()

# DFS depth first search
def solve_map_with_stack(x1, y1, x2, y2,k,isPath):

    img = cv2.imread(dataCSV[16])
    if k != -1:
        img = cv2.imread(dataCSV[k+9])
        img[x1-1][y1-1]=[222,86,73]

    stack = []
    stack.append((x1, y1))
    Map[x1][y1] = 4  # 表示已经走過的路

    while len(stack) > 0:
        cur_node = stack[-1]
        # print(cur_node)
        if cur_node == (x2, y2):

            break
        for d in dirs:
            next_x, next_y = d(*cur_node)
            # print(next_x, next_y )
            if Map[next_x][next_y] == 0 or Map[next_x][next_y] == 3:
                stack.append((next_x, next_y))
                pathSet.add((next_x,next_y))
                path.append((next_x,next_y))
                Map[next_x][next_y] = 4
                if k != -1:
                    img[next_x-1][next_y-1]=[222,86,73]
                break
        else:
            stack.pop()

    if k != -1 and isPath == 0:
        cv2.imwrite("Coverage_"+dataCSV[k][38:]+".jpg",img)
        files.download("Coverage_"+dataCSV[k][38:]+".jpg")

    if isPath == 1:

        cv2.imwrite("Path_"+dataCSV[k][38:]+".jpg",img)
        files.download("Path_"+dataCSV[k][38:]+".jpg")

    return stack


def printCoverage(pathSet,zeroList):
    # 使用正方形去計算
    print("使用正方形去計算Coverage 可達到： ","100" if (len(pathSet)>=(len(zeroList)+1)) else (len(pathSet)/(len(zeroList)+1)*100)," %")
    # 使用圓形去計算
    print("使用圓形去計算Coverage 可達到： ",(len(pathSet)*0.5*0.5*math.pi/(len(zeroList)+1)*100)," %")

def printPath(path):
    # print("全部走完的路徑 :",path)
    print("步數:",len(path))

def turnCount(path):
    turn =0 
    for i in range(0,len(path)-2):
        if path[i+2][0]-path[i+1][0] != path[i+1][0]-path[i][0] and path[i+2][1]-path[i+1][1] != path[i+1][1]-path[i][1]:
            turn +=1
    print("轉動次數為：",turn)
    print("自定義成本(轉動+步數)",turn+len(path))


for k in range(9):
    originMap = np.array(pd.read_csv(dataCSV[k]))

    Map= originMap

    start=[]
    end=[]
    zeroList=[]
    path=[]
    pathSet = set()

    # padding 
    Map=np.pad(originMap, ((1, 1), (1, 2)), 'constant', constant_values=1)

    # find start and end and zero
    for i in range(len(Map)):
        for j in range(len(Map[0])):
            if Map[i][j]==0:
                zeroList.append([i,j])
            if Map[i][j]==2:
                start =[i,j]
            if Map[i][j]==3:
                end =[i,j]

    '''設定一個不能到的點就能全部走完'''
    solve_map_with_stack(start[0], start[1], 0, 0,k,0)

    #還原地圖 
    Map=np.pad(originMap, ((1, 1), (1, 2)), 'constant', constant_values=1)

    # 從路經中的最後一個點當成新的起點
    stack = solve_map_with_stack(path[len(path)-1][0], path[len(path)-1][1], end[0], end[1],-1,0)
    print("Filename: ", dataCSV[k][38:])
    print("全局覆蓋")
    printCoverage(pathSet,zeroList)
    printPath(path)
    turnCount(path)

    #還原地圖 
    Map=np.pad(originMap, ((1, 1), (1, 2)), 'constant', constant_values=1)
    
    # 下面是最短路徑只有起點到終點
    path=[]
    pathSet = set()
    solve_map_with_stack(start[0], start[1], end[0], end[1],k,1)
    print("以下是最短路徑只有起點到終點")
    printCoverage(pathSet,zeroList)
    printPath(path)
    turnCount(path)
    print("------------------------------------------")



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Filename:  small-easy.csv
全局覆蓋
使用正方形去計算Coverage 可達到：  100  %
使用圓形去計算Coverage 可達到：  78.53981633974483  %
步數: 229126
轉動次數為： 3412
自定義成本(轉動+步數) 232538


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

Filename:  big-hard.csv
全局覆蓋
使用正方形去計算Coverage 可達到：  100  %
使用圓形去計算Coverage 可達到：  78.53981633974483  %
步數: 1509730
轉動次數為： 65430
自定義成本(轉動+步數) 1575160


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

以下是最短路徑只有起點到終點
使用正方形去計算Coverage 可達到：  0.4593100806618072  %
使用圓形去計算Coverage 可達到：  0.36074129378171715  %
步數: 6787
轉動次數為： 326
自定義成本(轉動+步數) 7113
------------------------------------------
